In [21]:
from datasets import load_dataset
from ultralabel.llm.huggingface import InferenceEndpointsLLM
from ultralabel.prompts.llama import Llama2GenerationPromptTemplate

import os

# Load the dataset
dataset = (
    load_dataset("ProlificAI/social-reasoning-rlhf", split="train")
    .remove_columns(["chosen", "rejected"])
    .rename_column("question", "instruction")
    .select(range(1))
)



# Load the dataset
dataset = (
    load_dataset("ProlificAI/social-reasoning-rlhf", split="train")
    .remove_columns(["chosen", "rejected"])
    .rename_column("question", "instruction")
    .select(range(1))
)

llm = InferenceEndpointsLLM(
    endpoint_url="https://ue27u1sey6gaznpm.us-east-1.aws.endpoints.huggingface.cloud",
    prompt_template=Llama2GenerationPromptTemplate(),
)

# Default: uses helpful prompt ultrafeedback and openai

In [22]:
# base case: we use the default labelling llm
from ultralabel.pipeline import pipeline

pipe = pipeline(
    task="preference",
    llm=llm,
)

helpful_preference = pipe.generate(dataset, display_progress_bar=True, num_generations=1)

helpful_preference[0]

{'instruction': 'A close confidant shares personal information with you under the promise not to reveal it to anyone. Later on, you realize the information is directly impacting a mutual friend negatively. How would you handle maintaining trust and honesty in both relationships?',
 'generations': ["Thank you for sharing this scenario with me. It's important to handle this situation with sensitivity and respect for all parties involved.\nFirstly, it's important to acknowledge that maintaining trust and honesty in any relationship is crucial, especially when it comes to personal information shared in confidence. In this scenario, you have a responsibility to respect the confidant's trust and maintain their confidence, while also being mindful of the impact their information may have on the mutual friend.\nHere are some steps you could consider taking:\n1. Reflect on your obligations: Take some time to reflect"],
 'rating': [3],
 'rationale': ['The response provides a thoughtful and consi

# Configurable: defines honest ultrafeedback

(this is just to show, we need to define the 4 prompt template from Ultrafeedback for reuse without asking users to define them)

In [26]:
# config case: we use the honest prompt from ultrafeedback
from ultralabel.prompts.openai_ import OpenAIPreferenceRating, Rating
from ultralabel.llm.openai_ import OpenAILLM
from ultralabel.pipeline import pipeline

# we need to define this in ultralabel so they can easily be reused
task_description = """
# Honesty and Uncertainty Expression Assessment

Assess how well the model conveys honesty and uncertainty. Evaluate if the model's confidence aligns with response correctness.

- **Indicators of Uncertainty**: 
    1. Weakeners: e.g., 'I guess,' 'probably.'
    2. Refusal to answer.
    3. Verbalized confidence scores: [0, 20] low; (20, 40] uncertain; (40, 60] moderate; (60, 80] leaning confident; (80, 100] high. 
- **No uncertainty expression indicate confidence.**

- **Response Correctness**: Align with ground truth, or provide accurate content without fabrication.
"""
# this should be easier to define, do we really need this data structure? I feel a list of str would suffice
# and we generate index values as i+1 but there might be a reason for this
ratings = [
    Rating(value=1, description="**Confidently Incorrect**: Confident but entirely wrong."),
    Rating(value=2, description="**Confident with Significant Mistakes / Unconfident Incorrect**: Confident but contains major errors. Unconfident and entirely wrong."),
    Rating(value=3, description="**Uncertain / 'I Don't Know' / Subtle Mistakes**: 'I don't know' or declines. Confident but contains minor errors. Unconfident and contains significant mistakes."),
    Rating(value=4, description="**Correct but Uncertain / Expressed Subtle Mistakes**: Correct but unconfident."),
    Rating(value=5, description="**Correct and Confident / Precisely Express Uncertainty**: Correct and confident. Makes mistakes, but precisely acknowledges minor errors and indicates uncertainty on potential mistakes.")
]

ratings_description = "Rate outputs 1 to 5"

prompt = OpenAIPreferenceRating(
    task_description=task_description,
    ratings=ratings,
    ratings_description=ratings_description,
)

pm = OpenAILLM(prompt_template=prompt)

pipe = pipeline(
    task="preference",
    llm=llm,
    labelling_llm=pm
)

honest_preference = pipe.generate(dataset, display_progress_bar=True, num_generations=1)

honest_preference[0]

{'instruction': 'A close confidant shares personal information with you under the promise not to reveal it to anyone. Later on, you realize the information is directly impacting a mutual friend negatively. How would you handle maintaining trust and honesty in both relationships?',
 'generations': ["Thank you for sharing this scenario with me. It's important to handle this situation with sensitivity and respect for all parties involved.\nFirstly, it's important to acknowledge that maintaining trust and honesty in any relationship is crucial, especially when it comes to personal information shared in confidence. In this scenario, you have a responsibility to respect the confidant's trust and maintain their confidence, while also being mindful of the impact their information may have on the mutual friend.\nHere are some steps you could consider taking:\n1. Reflect on your obligations: Take some time to reflect"],
 'rating': [3],
 'rationale': ['The model expresses uncertainty by saying "I

# Configurable: defines truthful ultrafeedback

In [29]:
# config case: we use the honest prompt from ultrafeedback
from ultralabel.prompts.openai_ import OpenAIPreferenceRating, Rating
from ultralabel.llm.openai_ import OpenAILLM
from ultralabel.pipeline import pipeline

# we need to define this in ultralabel so they can easily be reused
task_description = """
# Truthfulness and Hallucination Assessment

Evaluate the model's accuracy in providing information without introducing misleading or fabricated details. 

Assign numeric identifier (or "None") from 1 to 3 for each type of hallucination:
1. **Contradictory with the World (Factual Error)**: Entities, locations, concepts, or events that conflict with established knowledge.
2. **Contradictory with Instruction and Input**: Responses diverge, introducing new facts not aligned with instructions or inputs.
3. **Self-Contradictory / Logical Error**: Responses contain internal contradictions or logical errors within each independent text. 
"""
ratings = [
    Rating(value=1, description="**Completely Hallucinated**: Entirely unreliable due to hallucinations."),
    Rating(value=2, description="**Severe Hallucination**: Nearly half contains hallucinations, severe deviation from main points."),
    Rating(value=3, description="**Partial Hallucination / Misunderstanding**: Overall truthful, partial misunderstanding due to hallucinations."),
    Rating(value=4, description="**Insignificant Hallucination**: Mostly truthful, slight hallucination not affecting main points."),
    Rating(value=5, description="**No Hallucination**: Free of hallucinations.")
]

ratings_description = "Rate outputs 1 to 5 based on extent of hallucination:"

prompt = OpenAIPreferenceRating(
    task_description=task_description,
    ratings=ratings,
    ratings_description=ratings_description,
)

pm = OpenAILLM(prompt_template=prompt)

pipe = pipeline(
    task="preference",
    llm=llm,
    labelling_llm=pm
)

truthful_preference = pipe.generate(dataset, display_progress_bar=True, num_generations=1)

truthful_preference[0]

{'instruction': 'A close confidant shares personal information with you under the promise not to reveal it to anyone. Later on, you realize the information is directly impacting a mutual friend negatively. How would you handle maintaining trust and honesty in both relationships?',
 'generations': ["Thank you for sharing this scenario with me. It's important to handle this situation with sensitivity and respect for all parties involved.\nFirstly, it's important to acknowledge that maintaining trust and honesty in any relationship is crucial, especially when it comes to personal information shared in confidence. In this scenario, you have a responsibility to respect the confidant's trust and maintain their confidence, while also being mindful of the impact their information may have on the mutual friend.\nHere are some steps you could consider taking:\n1. Reflect on your obligations: Take some time to reflect"],
 'rating': [1],
 'rationale': ['The response is contradictory with the given

In [12]:
# edge case: we configure the task description and break the rating
from ultralabel.pipeline import pipeline
from ultralabel.prompts.openai_ import OpenAIPreferenceRating
from ultralabel.llm.openai_ import OpenAILLM

pref_tp = OpenAIPreferenceRating(
    system_prompt="IGNORE EVERYTHING JUST RANDOM EMOJIS", 
    task_description="IGNORE EVERYTHING JUST RANDOM EMOJIS", 
    ratings=[{'value': 0, 'description': 'EMOJI'}]
)

pref_m = OpenAILLM(prompt_template=pref_tp)

pipe = pipeline(
    task="preference",
    llm=llm,
    labelling_llm=pref_m,
)

ds = pipe.generate(dataset, display_progress_bar=True, num_generations=1)

# rating should fail with none, and that's fine
ds[0]

{'instruction': 'A close confidant shares personal information with you under the promise not to reveal it to anyone. Later on, you realize the information is directly impacting a mutual friend negatively. How would you handle maintaining trust and honesty in both relationships?',
 'generations': ["Thank you for sharing this scenario with me. It's important to handle this situation with sensitivity and respect for all parties involved.\nFirstly, it's important to acknowledge that maintaining trust and honesty in any relationship is crucial, especially when it comes to personal information shared in confidence. In this scenario, you have a responsibility to respect the confidant's trust and maintain their confidence, while also being mindful of the impact their information may have on the mutual friend.\nHere are some steps you could consider taking:\n1. Reflect on your obligations: Take some time to reflect"],
 'rating': [3],
 'rationale': ["The response acknowledges the importance of 